In [1]:
!pip install pandas
!pip install scikit-surprise


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict
from surprise import KNNWithMeans
from surprise import SVD, SVDpp
from surprise import KNNBaseline
from surprise import KNNBasic
from surprise import KNNWithZScore
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV

import time

In [3]:
start_time = time.time()
df = pd.read_csv("Feviews2.csv", names=["id","ProductId","Brand", "Color","ProductName","UserId","Score","Age","Gender","State"])
df.head()
computational_time = time.time() - start_time
print('Done in %0.3fs' %(computational_time))

Done in 0.455s


/tmp/ipykernel_11294/1772505102.py:2: DtypeWarning: Columns (6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Feviews2.csv", names=["id","ProductId","Brand", "Color","ProductName","UserId","Score","Age","Gender","State"])


In [4]:
df

,id,ProductId,Brand,Color,ProductName,UserId,Score,Age,Gender,State
0,NaN,ProductId,Brand,Color,ProductName,UserId,Score,Age,Gender,State
1,1.0,B00813GRG4,Gucci,brown,clock,A1D87F6ZCVE5NK,1,77,0,Bihar
2,2.0,B000LQOCH0,Samsung,pink,jacket,ABXLMWJIXXAIN,4,75,0,Tamil Nadu
3,3.0,B000UA0QIQ,Prada,blue,jacket,A395BORC6FGVXV,2,22,1,Himachal Pradesh
4,4.0,B006K2ZZ7K,Honda,grey,laptopsleeve,A1UQRSCLF8GW1T,5,12,0,Meghalaya
...,...,...,...,...,...,...,...,...,...,...
299997,299997.0,B000VA5LZI,Apple,grey,pen,A3NRNASNTU5TFI,5.0,29.0,0.0,Uttarakhand
299998,299998.0,B0007XAUXC,H&M,pink,diary,AYRFQU12QD60L,5.0,35.0,0.0,Gujarat
299999,299999.0,B0007XAUXC,H&M,pink,diary,A1TDZMXV5P6AJB,4.0,53.0,0.0,Telangana
300000,300000.0,B0007XAUXC,H&M,pink,diary,A2IVZ6LY3YHV1N,5.0,53.0,1.0,Arunachal Pradesh


In [5]:
df.dtypes

id             float64
ProductId       object
Brand           object
Color           object
ProductName     object
UserId          object
Score           object
Age             object
Gender          object
State           object
dtype: object

In [6]:
rows_count, columns_count = df.shape
print('Total Number of rows :', rows_count)
print('Total Number of columns :', columns_count)

Total Number of rows : 300002
Total Number of columns : 10


In [7]:
df.dtypes

id             float64
ProductId       object
Brand           object
Color           object
ProductName     object
UserId          object
Score           object
Age             object
Gender          object
State           object
dtype: object

In [8]:
unique_userId = df['UserId'].nunique()
unique_productId = df['ProductId'].nunique()
print('Total number of unique Users    : ', unique_userId)
print('Total number of unique Products : ', unique_productId)

Total number of unique Users    :  165097
Total number of unique Products :  39016


In [9]:
df1 = df.dropna()

In [10]:
df1.apply(lambda x : sum(x.isnull()))

id             0
ProductId      0
Brand          0
Color          0
ProductName    0
UserId         0
Score          0
Age            0
Gender         0
State          0
dtype: int64

In [11]:
df1

,id,ProductId,Brand,Color,ProductName,UserId,Score,Age,Gender,State
1,1.0,B00813GRG4,Gucci,brown,clock,A1D87F6ZCVE5NK,1,77,0,Bihar
2,2.0,B000LQOCH0,Samsung,pink,jacket,ABXLMWJIXXAIN,4,75,0,Tamil Nadu
3,3.0,B000UA0QIQ,Prada,blue,jacket,A395BORC6FGVXV,2,22,1,Himachal Pradesh
4,4.0,B006K2ZZ7K,Honda,grey,laptopsleeve,A1UQRSCLF8GW1T,5,12,0,Meghalaya
5,5.0,B006K2ZZ7K,Honda,grey,laptopsleeve,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh
...,...,...,...,...,...,...,...,...,...,...
299996,299996.0,B000VA5LZI,Apple,grey,pen,AE6YV1KF6UKJR,2.0,63.0,0.0,Madhya Pradesh
299997,299997.0,B000VA5LZI,Apple,grey,pen,A3NRNASNTU5TFI,5.0,29.0,0.0,Uttarakhand
299998,299998.0,B0007XAUXC,H&M,pink,diary,AYRFQU12QD60L,5.0,35.0,0.0,Gujarat
299999,299999.0,B0007XAUXC,H&M,pink,diary,A1TDZMXV5P6AJB,4.0,53.0,0.0,Telangana


In [12]:
df_transpose = df1.describe().T
df_transpose

,count,mean,std,min,25%,50%,75%,max
id,300000.0,150000.5,86602.684716,1.0,75000.75,150000.5,225000.25,300000.0


In [13]:
df1['Score'].value_counts()

Score
5    148807
5     40939
4     33756
1     21580
3     17926
2     12396
4      9501
1      6205
3      5238
2      3652
Name: count, dtype: int64

In [14]:
def map_score(score):
    if isinstance(score, str) and score.lower() == 'score':
        return score
    return int(float(score))
df1['Mapped_Score'] = df1['Score'].map(map_score)


/tmp/ipykernel_11294/1642850552.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Mapped_Score'] = df1['Score'].map(map_score)


In [15]:
df1['Mapped_Score'].value_counts()


Mapped_Score
5    189746
4     43257
1     27785
3     23164
2     16048
Name: count, dtype: int64

In [16]:
df1

,id,ProductId,Brand,Color,ProductName,UserId,Score,Age,Gender,State,Mapped_Score
1,1.0,B00813GRG4,Gucci,brown,clock,A1D87F6ZCVE5NK,1,77,0,Bihar,1
2,2.0,B000LQOCH0,Samsung,pink,jacket,ABXLMWJIXXAIN,4,75,0,Tamil Nadu,4
3,3.0,B000UA0QIQ,Prada,blue,jacket,A395BORC6FGVXV,2,22,1,Himachal Pradesh,2
4,4.0,B006K2ZZ7K,Honda,grey,laptopsleeve,A1UQRSCLF8GW1T,5,12,0,Meghalaya,5
5,5.0,B006K2ZZ7K,Honda,grey,laptopsleeve,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh,4
...,...,...,...,...,...,...,...,...,...,...,...
299996,299996.0,B000VA5LZI,Apple,grey,pen,AE6YV1KF6UKJR,2.0,63.0,0.0,Madhya Pradesh,2
299997,299997.0,B000VA5LZI,Apple,grey,pen,A3NRNASNTU5TFI,5.0,29.0,0.0,Uttarakhand,5
299998,299998.0,B0007XAUXC,H&M,pink,diary,AYRFQU12QD60L,5.0,35.0,0.0,Gujarat,5
299999,299999.0,B0007XAUXC,H&M,pink,diary,A1TDZMXV5P6AJB,4.0,53.0,0.0,Telangana,4


In [17]:
rating_counts = pd.DataFrame(df1['Mapped_Score'].value_counts()).reset_index()
rating_counts.columns = ['Labels', 'Ratings']
rating_counts

,Labels,Ratings
0,5,189746
1,4,43257
2,1,27785
3,3,23164
4,2,16048


In [18]:
df1

,id,ProductId,Brand,Color,ProductName,UserId,Score,Age,Gender,State,Mapped_Score
1,1.0,B00813GRG4,Gucci,brown,clock,A1D87F6ZCVE5NK,1,77,0,Bihar,1
2,2.0,B000LQOCH0,Samsung,pink,jacket,ABXLMWJIXXAIN,4,75,0,Tamil Nadu,4
3,3.0,B000UA0QIQ,Prada,blue,jacket,A395BORC6FGVXV,2,22,1,Himachal Pradesh,2
4,4.0,B006K2ZZ7K,Honda,grey,laptopsleeve,A1UQRSCLF8GW1T,5,12,0,Meghalaya,5
5,5.0,B006K2ZZ7K,Honda,grey,laptopsleeve,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh,4
...,...,...,...,...,...,...,...,...,...,...,...
299996,299996.0,B000VA5LZI,Apple,grey,pen,AE6YV1KF6UKJR,2.0,63.0,0.0,Madhya Pradesh,2
299997,299997.0,B000VA5LZI,Apple,grey,pen,A3NRNASNTU5TFI,5.0,29.0,0.0,Uttarakhand,5
299998,299998.0,B0007XAUXC,H&M,pink,diary,AYRFQU12QD60L,5.0,35.0,0.0,Gujarat,5
299999,299999.0,B0007XAUXC,H&M,pink,diary,A1TDZMXV5P6AJB,4.0,53.0,0.0,Telangana,4


In [19]:
df1 = df1.drop(df1[df1['Score'] == 'Score'].index)
df1['Mapped_Score'].value_counts()


Mapped_Score
5    189746
4     43257
1     27785
3     23164
2     16048
Name: count, dtype: int64

In [20]:
rating_counts = pd.DataFrame(df1['Mapped_Score'].value_counts()).reset_index()
rating_counts.columns = ['Labels', 'Ratings']
rating_counts

,Labels,Ratings
0,5,189746
1,4,43257
2,1,27785
3,3,23164
4,2,16048


In [22]:
df1.corr()

ValueError: could not convert string to float: 'B00813GRG4'

In [23]:
df1.head()

,id,ProductId,Brand,Color,ProductName,UserId,Score,Age,Gender,State,Mapped_Score
1,1.0,B00813GRG4,Gucci,brown,clock,A1D87F6ZCVE5NK,1,77,0,Bihar,1
2,2.0,B000LQOCH0,Samsung,pink,jacket,ABXLMWJIXXAIN,4,75,0,Tamil Nadu,4
3,3.0,B000UA0QIQ,Prada,blue,jacket,A395BORC6FGVXV,2,22,1,Himachal Pradesh,2
4,4.0,B006K2ZZ7K,Honda,grey,laptopsleeve,A1UQRSCLF8GW1T,5,12,0,Meghalaya,5
5,5.0,B006K2ZZ7K,Honda,grey,laptopsleeve,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh,4


In [24]:
df1


,id,ProductId,Brand,Color,ProductName,UserId,Score,Age,Gender,State,Mapped_Score
1,1.0,B00813GRG4,Gucci,brown,clock,A1D87F6ZCVE5NK,1,77,0,Bihar,1
2,2.0,B000LQOCH0,Samsung,pink,jacket,ABXLMWJIXXAIN,4,75,0,Tamil Nadu,4
3,3.0,B000UA0QIQ,Prada,blue,jacket,A395BORC6FGVXV,2,22,1,Himachal Pradesh,2
4,4.0,B006K2ZZ7K,Honda,grey,laptopsleeve,A1UQRSCLF8GW1T,5,12,0,Meghalaya,5
5,5.0,B006K2ZZ7K,Honda,grey,laptopsleeve,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh,4
...,...,...,...,...,...,...,...,...,...,...,...
299996,299996.0,B000VA5LZI,Apple,grey,pen,AE6YV1KF6UKJR,2.0,63.0,0.0,Madhya Pradesh,2
299997,299997.0,B000VA5LZI,Apple,grey,pen,A3NRNASNTU5TFI,5.0,29.0,0.0,Uttarakhand,5
299998,299998.0,B0007XAUXC,H&M,pink,diary,AYRFQU12QD60L,5.0,35.0,0.0,Gujarat,5
299999,299999.0,B0007XAUXC,H&M,pink,diary,A1TDZMXV5P6AJB,4.0,53.0,0.0,Telangana,4


In [25]:
df1 = df1.drop(['HelpfulnessNumerator'], axis=1)
df1 = df1.drop(['HelpfulnessDenominator'], axis=1)
df1 = df1.drop(['Summary'],axis = 1)
df1 = df1.drop(['Text'],axis = 1)
df1 = df1.drop(['Score'],axis = 1)

KeyError: "['HelpfulnessNumerator'] not found in axis"

In [26]:
df1.head()


,id,ProductId,Brand,Color,ProductName,UserId,Score,Age,Gender,State,Mapped_Score
1,1.0,B00813GRG4,Gucci,brown,clock,A1D87F6ZCVE5NK,1,77,0,Bihar,1
2,2.0,B000LQOCH0,Samsung,pink,jacket,ABXLMWJIXXAIN,4,75,0,Tamil Nadu,4
3,3.0,B000UA0QIQ,Prada,blue,jacket,A395BORC6FGVXV,2,22,1,Himachal Pradesh,2
4,4.0,B006K2ZZ7K,Honda,grey,laptopsleeve,A1UQRSCLF8GW1T,5,12,0,Meghalaya,5
5,5.0,B006K2ZZ7K,Honda,grey,laptopsleeve,ADT0SRK1MGOEU,4,33,0,Madhya Pradesh,4


In [27]:
users_counts = df1['UserId'].value_counts().rename('users_counts')
users_data   = df1.merge(users_counts.to_frame(),
                                left_on='UserId',
                                right_index=True)

In [28]:
subset_df = users_data[users_data.users_counts>=10]
subset_df

,id,ProductId,Brand,Color,ProductName,UserId,Score,Age,Gender,State,Mapped_Score,users_counts
13,13.0,B001GVISJM,Coca-Cola,brown,laptopsleeve,A18ECVX2RJ7HUE,4,36,1,Arunachal Pradesh,4,12
23194,23194.0,B004ZIER34,Honda,brown,skirt,A18ECVX2RJ7HUE,3,36,1,Arunachal Pradesh,3,12
33370,33370.0,B007JFMH96,Toyota,brown,jeans,A18ECVX2RJ7HUE,3,36,1,Arunachal Pradesh,3,12
45921,45921.0,B0041NYV8E,BMW,black,jacket,A18ECVX2RJ7HUE,5,36,1,Arunachal Pradesh,5,12
70299,70299.0,B007I7Z3Z0,BMW,pink,watch,A18ECVX2RJ7HUE,5,36,1,Arunachal Pradesh,5,12
...,...,...,...,...,...,...,...,...,...,...,...,...
205331,205331.0,B001BORBHO,Louis Vuitton,orange,pen,A163A92N8CQI7G,5,57,0,Rajasthan,5,10
216050,216050.0,B001BORBMY,Coca-Cola,purple,jacket,A163A92N8CQI7G,5,57,0,Rajasthan,5,10
216081,216081.0,B001BORBMY,Coca-Cola,purple,jacket,A163A92N8CQI7G,5,57,0,Rajasthan,5,10
216197,216197.0,B001BORBM4,Zara,green,bottle,A163A92N8CQI7G,5,57,0,Rajasthan,5,10


In [29]:
product_rating_counts = subset_df['ProductId'].value_counts().rename('product_rating_counts')
product_rating_data   = subset_df.merge(product_rating_counts.to_frame(),
                                left_on='ProductId',
                                right_index=True)
product_rating_data = product_rating_data[product_rating_data.product_rating_counts >= 10]
product_rating_data

,id,ProductId,Brand,Color,ProductName,UserId,Score,Age,Gender,State,Mapped_Score,users_counts,product_rating_counts
23194,23194.0,B004ZIER34,Honda,brown,skirt,A18ECVX2RJ7HUE,3,36,1,Arunachal Pradesh,3,12,156
23077,23077.0,B004ZIER34,Honda,brown,skirt,A1Z54EM24Y40LL,4,43,1,Tripura,4,142,156
23149,23149.0,B004ZIER34,Honda,brown,skirt,A19SDJ2TSSL8HI,3,40,0,Uttar Pradesh,3,31,156
23027,23027.0,B004ZIER34,Honda,brown,skirt,A22PUBSSNP54L,4,54,0,Odisha,4,34,156
23059,23059.0,B004ZIER34,Honda,brown,skirt,ACYR6O588USK,5,30,0,Karnataka,5,68,156
...,...,...,...,...,...,...,...,...,...,...,...,...,...
257871,257871.0,B00125ZCWI,L'Oreal,black,watch,ATBRBWM00I3K8,5,44,1,Nagaland,5,10,12
257875,257875.0,B00125ZCWI,L'Oreal,black,watch,A3TFNYQLGTH4NJ,5,67,0,Madhya Pradesh,5,10,12
257935,257935.0,B00125ZCWI,L'Oreal,black,watch,A3TFNYQLGTH4NJ,5,67,0,Madhya Pradesh,5,10,12
257917,257917.0,B00125ZCWI,L'Oreal,black,watch,A2N3N439PRGV3I,5,8,0,Meghalaya,5,10,12


In [30]:
product_rating_data

,id,ProductId,Brand,Color,ProductName,UserId,Score,Age,Gender,State,Mapped_Score,users_counts,product_rating_counts
23194,23194.0,B004ZIER34,Honda,brown,skirt,A18ECVX2RJ7HUE,3,36,1,Arunachal Pradesh,3,12,156
23077,23077.0,B004ZIER34,Honda,brown,skirt,A1Z54EM24Y40LL,4,43,1,Tripura,4,142,156
23149,23149.0,B004ZIER34,Honda,brown,skirt,A19SDJ2TSSL8HI,3,40,0,Uttar Pradesh,3,31,156
23027,23027.0,B004ZIER34,Honda,brown,skirt,A22PUBSSNP54L,4,54,0,Odisha,4,34,156
23059,23059.0,B004ZIER34,Honda,brown,skirt,ACYR6O588USK,5,30,0,Karnataka,5,68,156
...,...,...,...,...,...,...,...,...,...,...,...,...,...
257871,257871.0,B00125ZCWI,L'Oreal,black,watch,ATBRBWM00I3K8,5,44,1,Nagaland,5,10,12
257875,257875.0,B00125ZCWI,L'Oreal,black,watch,A3TFNYQLGTH4NJ,5,67,0,Madhya Pradesh,5,10,12
257935,257935.0,B00125ZCWI,L'Oreal,black,watch,A3TFNYQLGTH4NJ,5,67,0,Madhya Pradesh,5,10,12
257917,257917.0,B00125ZCWI,L'Oreal,black,watch,A2N3N439PRGV3I,5,8,0,Meghalaya,5,10,12


In [31]:
amazon_df = product_rating_data.copy()

In [32]:
panda_data = amazon_df.drop(['users_counts', 'product_rating_counts'], axis=1)

In [33]:
panda_data.head()

,id,ProductId,Brand,Color,ProductName,UserId,Score,Age,Gender,State,Mapped_Score
23194,23194.0,B004ZIER34,Honda,brown,skirt,A18ECVX2RJ7HUE,3,36,1,Arunachal Pradesh,3
23077,23077.0,B004ZIER34,Honda,brown,skirt,A1Z54EM24Y40LL,4,43,1,Tripura,4
23149,23149.0,B004ZIER34,Honda,brown,skirt,A19SDJ2TSSL8HI,3,40,0,Uttar Pradesh,3
23027,23027.0,B004ZIER34,Honda,brown,skirt,A22PUBSSNP54L,4,54,0,Odisha,4
23059,23059.0,B004ZIER34,Honda,brown,skirt,ACYR6O588USK,5,30,0,Karnataka,5


In [34]:
# product_name = panda_data[df['ProductId'] == '']['ProductName'].values[0]


In [35]:
k = 10
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(panda_data[['UserId', 'ProductId', 'Mapped_Score']], reader)

In [36]:
trainset, testset = train_test_split(surprise_data, test_size=.30, random_state=42)

In [37]:
start_time = time.time()
svd_model = SVD(n_epochs=20, lr_all=0.005, reg_all=0.2)
svd_model.fit(trainset)
predictions_svd = svd_model.test(testset)
svd_cv = cross_validate(svd_model, surprise_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
svd_df = pd.DataFrame.from_dict(svd_cv)
svd_described = svd_df.describe()
cv_results = pd.DataFrame([['SVD', svd_described['test_rmse']['mean'], svd_described['test_mae']['mean'],
                           svd_described['fit_time']['mean'], svd_described['test_time']['mean']]],
                            columns = ['Model', 'RMSE', 'MAE', 'Fit Time', 'Test Time'])


# get RMSE
print("\n\n==================== Model Evaluation ===============================")
accuracy.rmse(predictions_svd, verbose=True)
print("=====================================================================")
computational_time = time.time() - start_time
print('\n Computational Time : %0.3fs' %(computational_time))
cv_results

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8688  0.9051  0.8796  0.8895  0.8847  0.8855  0.0120  
MAE (testset)     0.6691  0.6909  0.6805  0.6806  0.6821  0.6806  0.0069  
Fit time          0.16    0.16    0.16    0.16    0.17    0.16    0.00    
Test time         0.03    0.03    0.03    0.03    0.05    0.03    0.01    


==================== Model Evaluation ===============================
RMSE: 0.9117

 Computational Time : 1.345s


,Model,RMSE,MAE,Fit Time,Test Time
0,SVD,0.885539,0.68064,0.158178,0.031469


In [40]:
start_time = time.time()
svdpp_model = SVDpp(n_epochs=25, lr_all=0.01, reg_all=0.4)
svdpp_model.fit(trainset)
predictions_svdpp = svdpp_model.test(testset)
svdpp_cv = cross_validate(svdpp_model, surprise_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
svdpp_df = pd.DataFrame.from_dict(svdpp_cv)
svdpp_described = svdpp_df.describe()
svdpp_cv_results = pd.DataFrame([['SVDpp', svdpp_described['test_rmse']['mean'], svdpp_described['test_mae']['mean'],
                           svdpp_described['fit_time']['mean'], svdpp_described['test_time']['mean']]],
                            columns = ['Model', 'RMSE', 'MAE', 'Fit Time', 'Test Time'])

cv_results = cv_results._append(svdpp_cv_results, ignore_index=True)
print("\n\n==================== Model Evaluation ===============================")
accuracy.rmse(predictions_svdpp, verbose=True)
print("=====================================================================")
computational_time = time.time() - start_time
print('\n Computational Time : %0.3fs' %(computational_time))
cv_results

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8642  0.8812  0.8579  0.8505  0.8640  0.8636  0.0101  
MAE (testset)     0.6521  0.6631  0.6568  0.6453  0.6582  0.6551  0.0060  
Fit time          0.74    0.77    0.72    0.64    0.62    0.70    0.06    
Test time         0.10    0.11    0.11    0.10    0.11    0.11    0.00    


==================== Model Evaluation ===============================
RMSE: 0.8778

 Computational Time : 5.146s


,Model,RMSE,MAE,Fit Time,Test Time
0,SVD,0.885539,0.680640,0.158178,0.031469
1,SVDpp,0.863557,0.655107,0.697939,0.106092


In [ ]:
predictions_svd

[Prediction(uid='A3NIDDT7E7JIFW', iid='B006F2NYI2', r_ui=1.0, est=4.548342096304112, details={'was_impossible': False}),
 Prediction(uid='A17GK9E70O7Y9R', iid='B008Z4VAPM', r_ui=3.0, est=3.552137718336177, details={'was_impossible': False}),
 Prediction(uid='AO2MVU9UIR2BM', iid='B00474HDWS', r_ui=1.0, est=3.8716933958702615, details={'was_impossible': False}),
 Prediction(uid='AIJXKCP2XE3MK', iid='B001CU0N7C', r_ui=5.0, est=4.174439955133264, details={'was_impossible': False}),
 Prediction(uid='A1QUXQA6WSVTH1', iid='B000ENUC3S', r_ui=5.0, est=4.391906453113001, details={'was_impossible': False}),
 Prediction(uid='A31BG4GILLHMQP', iid='B001D0KG4Q', r_ui=3.0, est=3.970968462797069, details={'was_impossible': False}),
 Prediction(uid='AP62C8V1SLZSG', iid='B001E5DWW8', r_ui=5.0, est=4.611321198787846, details={'was_impossible': False}),
 Prediction(uid='A2C0ZF6J18T9WV', iid='B001CD1VI4', r_ui=5.0, est=4.216669470734212, details={'was_impossible': False}),
 Prediction(uid='A1TMAVN4CEM8U8', 

In [41]:
top_n = defaultdict(list)
def get_top_n(predictions, n=k):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predictions_svd, n=k)
top_n

defaultdict(list,
            {'A1BJ67411QA0WZ': [('B00213IAUQ', 3.953348815729238),
              ('B00213ERI0', 3.7427899253652193)],
             'A1RFZGWX46N8H7': [('B000FDDESC', 4.134464401546693),
              ('B001LG945O', 3.264796667415764),
              ('B004E4CCSQ', 3.1932432389753664)],
             'A3PJZ8TU8FDQ1K': [('B000H7LVKY', 4.7555053624914905),
              ('B0051COPH6', 4.714278319434619),
              ('B008AV5HLS', 4.704512054217086),
              ('B000G6MBUA', 4.673870577764365),
              ('B0012XBD7I', 4.647080594848044),
              ('B0058AMY74', 4.60787487031501),
              ('B0058AMY74', 4.60787487031501),
              ('B000G6RYNE', 4.607338229518761),
              ('B0049ULB78', 4.569764530049242),
              ('B000LKXBL4', 4.562160809879601)],
             'A1LP1PGQLL9B5F': [('B0090X8IPM', 4.053133233541858)],
             'AQQLWCMRNDFGI': [('B000EQT574', 4.115912483850766),
              ('B001J9QBU4', 4.089538338803633),
      

In [42]:
formatted_data = {}
for user_id, ratings in top_n.items():
    formatted_ratings = [{"ProductId": product_id, "Rating": rating} for product_id, rating in ratings]
    formatted_data[user_id] = formatted_ratings

# Save the formatted data to a JSON file
with open('formatted_top_n.json', 'w') as json_file:
    json.dump(formatted_data, json_file, indent=4)


In [ ]:
start_time = time.time()
df = pd.read_csv("Reviews.csv", names=["Id","ProductId", "UserId","ProfileName","HelpfulnessNumerator","HelpfulnessDenominator","Score","Time","Summary","Text"])
df.head()
computational_time = time.time() - start_time
print('Done in %0.3fs' %(computational_time))

FileNotFoundError: ignored

In [ ]:
start_time = time.time()
df = pd.read_csv("Reviews.csv", names=["Id","ProductId", "UserId","ProfileName","HelpfulnessNumerator","HelpfulnessDenominator","Score","Time","Summary","Text"])
df.head()
computational_time = time.time() - start_time
print('Done in %0.3fs' %(computational_time))
df1 = df.dropna()
sample_size = 1000  # Adjust as needed
top_k_features = 500  # Adjust as needed

# Sample a subset of your data (assuming df1 is your DataFrame)
df2 = df1.sample(n=sample_size, random_state=42)
df2.index = range(1, len(df2) + 1)
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
df2['Summary'] = df2['Summary'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['Summary'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(df2.index, index=df2['ProductId']).drop_duplicates()
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df2['ProductId'].iloc[movie_indices]
get_recommendations('B001M2BM4I')

KeyError: ignored

In [ ]:
df1 = df.dropna()

In [ ]:
sample_size = 1000  # Adjust as needed
top_k_features = 500  # Adjust as needed

# Sample a subset of your data (assuming df1 is your DataFrame)
df2 = df1.sample(n=sample_size, random_state=42)


In [ ]:
df2

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
548708,548708,B001M2BM4I,A2OH0YZ2XEQA63,Maria Orfan,0,0,4,1309824000,Espresso at home.,"Description said extra, extra bold and it deli..."
300637,300637,B000AY9UIY,A22CW0ZHY3NJH8,Noname,1,1,4,1288224000,A great addition to the Mini-Wheats line,I am a huge fan of Kellogg's Frosted Mini-Whea...
418337,418337,B00267A348,A3F7UDB1JE5V21,M. Russell,2,3,5,1251763200,A different kind of tea,Quite refreshing. If you want something diffe...
550217,550217,B002Z613JO,A3CIAUOZM72H0C,justlikeschleck,1,1,5,1301529600,Great Beans!,I bought a pack of these beans to try what Kon...
500097,500097,B00061MRB8,A2MPBYQKMXHVN1,"Means Johnston III ""Means""",0,0,4,1238025600,Defurr-ums for Tinkerbell,Our Cat Tinkerbell loves cat treats (which cat...
...,...,...,...,...,...,...,...,...,...,...
220717,220717,B000EUG6YA,A1HG5OEMT3QPIP,Michelle P,0,0,5,1340755200,BEST COOKIES EVER!!!,I can't understand how these cookies could pos...
217187,217187,B000ESLJ6C,A2RWPLOHPUX8WB,pateo,0,0,5,1335484800,pero instant natural beverage,"<a href=""http://www.amazon.com/gp/product/B000..."
353984,353984,B0027049E4,A2V3FFFM6MV5OM,"B. Ogatiy ""bogatiy""",0,0,3,1339113600,Not a good substitute for Postum,I ordered this along with Cafix and Roastaroma...
434963,434963,B000FYXBHU,A3JA8EXHWOUV6M,Fish Head Soup,0,0,3,1334793600,Good but could be better.,Overall this brand is good but I think it is l...


In [ ]:
df2.index = range(1, len(df2) + 1)

In [ ]:
df2

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
1,1,B001M2BM4I,A2OH0YZ2XEQA63,Maria Orfan,0,0,4,1309824000,Espresso at home.,"Description said extra, extra bold and it deli..."
2,2,B000AY9UIY,A22CW0ZHY3NJH8,Noname,1,1,4,1288224000,A great addition to the Mini-Wheats line,I am a huge fan of Kellogg's Frosted Mini-Whea...
3,3,B00267A348,A3F7UDB1JE5V21,M. Russell,2,3,5,1251763200,A different kind of tea,Quite refreshing. If you want something diffe...
4,4,B002Z613JO,A3CIAUOZM72H0C,justlikeschleck,1,1,5,1301529600,Great Beans!,I bought a pack of these beans to try what Kon...
5,5,B00061MRB8,A2MPBYQKMXHVN1,"Means Johnston III ""Means""",0,0,4,1238025600,Defurr-ums for Tinkerbell,Our Cat Tinkerbell loves cat treats (which cat...
...,...,...,...,...,...,...,...,...,...,...
996,996,B000EUG6YA,A1HG5OEMT3QPIP,Michelle P,0,0,5,1340755200,BEST COOKIES EVER!!!,I can't understand how these cookies could pos...
997,997,B000ESLJ6C,A2RWPLOHPUX8WB,pateo,0,0,5,1335484800,pero instant natural beverage,"<a href=""http://www.amazon.com/gp/product/B000..."
998,998,B0027049E4,A2V3FFFM6MV5OM,"B. Ogatiy ""bogatiy""",0,0,3,1339113600,Not a good substitute for Postum,I ordered this along with Cafix and Roastaroma...
999,999,B000FYXBHU,A3JA8EXHWOUV6M,Fish Head Soup,0,0,3,1334793600,Good but could be better.,Overall this brand is good but I think it is l...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
df2['Summary'] = df2['Summary'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['Summary'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape


(1000, 1143)

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df2.index, index=df2['ProductId']).drop_duplicates()

In [ ]:
tfidf_matrix

<1000x1143 sparse matrix of type '<class 'numpy.float64'>'
	with 2686 stored elements in Compressed Sparse Row format>

In [ ]:
len(indices)

1000

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df2['ProductId'].iloc[movie_indices]

In [ ]:
get_recommendations('B001M2BM4I')

514    B001ELL9GI
549    B000UJREM8
260    B000H0ZJHW
14     B001RVFEP2
59     B0013A0QXC
300    B000E1S8NQ
386    B000EMB012
510    B000ETAJZI
867    B000MP1FW0
877    B00375LB80
Name: ProductId, dtype: object